In [1]:
#Data libraries:
import xarray as xr
import numpy as np

In [2]:
# Let's load Dask to make Xarray more efficient
import dask.distributed as dsk

# Start client
clnt = dsk.Client()
clnt

/g/data/hh5/public/apps/miniconda3/envs/analysis3-22.10/lib/python3.9/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 37595 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/37595/status,
Dashboard: /proxy/37595/status,Workers: 7
Total threads: 7,Total memory: 32.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:39579,Workers: 7
Dashboard: /proxy/37595/status,Total threads: 7
Started: Just now,Total memory: 32.00 GiB
Comm: tcp://127.0.0.1:44743,Total threads: 1
Dashboard: /proxy/37541/status,Memory: 4.57 GiB
Nanny: tcp://127.0.0.1:45941,


In [24]:
datadir = '/g/data/hh5/tmp/WACI-Hackathon-2023/AUS2200/data'
fname = 'accum_ls_prcp_10min_20220228.nc'
ds = xr.open_dataset(f'{datadir}/{fname}', chunks = {'time': -1, 'latitude':'200mb'})
ds

<xarray.Dataset>
Dimensions:        (time: 144, latitude: 2120, longitude: 2600)
Coordinates:
  * time           (time) datetime64[ns] 2022-02-28T00:05:00.000000256 ... 20...
  * latitude       (latitude) float32 -48.79 -48.77 -48.75 ... -6.852 -6.832
  * longitude      (longitude) float32 107.5 107.5 107.6 ... 158.9 159.0 159.0
Data variables:
    accum_ls_prcp  (time, latitude, longitude) float64 dask.array<chunksize=(144, 66, 2600), meta=np.ndarray>

In [25]:
pr_1hr = ds.accum_ls_prcp.resample(time='1H').sum()
pr_1hr.compute()

<xarray.DataArray 'accum_ls_prcp' (time: 24, latitude: 2120, longitude: 2600)>
array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
...
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]])
Coordinates:
  * latitude   (latitude) float32 -48.79 -48.77 -48.75 ... -6.871 -6.852 -6.832
  * longitude  (longitude) float32 107.5 107.5 107.6 107.6 ... 158.9 159.0 159.0
  * time       (time) datetime64[ns] 2022-02-28 ... 2022-02-28T23:00:00
Attributes:
    standard_name:  stratiform_rainfall_amount
    units:          kg m-2
    source:         Data from Met Office Unified Model
    um_version:     12.2
    STASH:          [  1   4 201]
    cell_methods:   time: mean (interval: 1 hour)

In [26]:
pr_1hr.to_netcdf('/g/data/ua8/Working/pxp581/pr_1hr_sum_20220228.nc')